In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.3
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:05:08)
SparkContext available as sc, HiveContext available as sqlContext.
Adding files: [/Users/pritykovskaya/development/graphframes-0.3.0-spark1.6-s_2.10.jar]


In [2]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

## lesson 3.1

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

graphData = sc.parallelize(edgeList).map(lambda (src, dst): Row(src, dst))

graphSchemaAB = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('B', StringType(), nullable=False)])

abDF = sqlCtx.createDataFrame(graphData, graphSchemaAB)

graphSchemaBC1 = StructType([StructField('B', IntegerType(), nullable=False),
                       StructField('C', StringType(), nullable=False)])
bcDF = sqlCtx.createDataFrame(graphData, graphSchemaBC1)

In [5]:
abDF.show()

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [5]:
bcDF.show()

+---+---+
|  B|  C|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [6]:
abcDF = abDF.join(bcDF, "B")
abcDF.show()

+---+---+---+
|  B|  A|  C|
+---+---+---+
|  3|  1|  4|
|  3|  1|  5|
|  3|  1|  6|
|  3|  1|  7|
|  3|  2|  4|
|  3|  2|  5|
|  3|  2|  6|
|  3|  2|  7|
|  2|  1|  3|
|  2|  1|  5|
+---+---+---+



In [7]:
abcDF = abDF.join(bcDF, "B")

abcDF.filter("A = 1").show()


+---+---+---+
|  B|  A|  C|
+---+---+---+
|  3|  1|  4|
|  3|  1|  5|
|  3|  1|  6|
|  3|  1|  7|
|  2|  1|  3|
|  2|  1|  5|
+---+---+---+



In [8]:
abcDF.drop("B") \
    .groupBy("A", "C") \
    .count() \
    .filter("A = 1") \
    .show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  1|  3|    1|
|  1|  4|    1|
|  1|  5|    2|
|  1|  6|    1|
|  1|  7|    1|
+---+---+-----+



## lesson 3.2

In [3]:
from graphframes import * 

vertices = sqlCtx.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("2", "1", "friend"),
  ("1", "3", "friend"),
  ("3", "1", "friend"),
  ("1", "4", "friend"),
  ("4", "1", "friend"),
  ("2", "3", "friend"), 
  ("3", "2", "friend"),
  ("2", "5", "friend"),
  ("5", "2", "friend"),
  ("3", "4", "friend"),
  ("4", "3", "friend"),
  ("3", "5", "friend"),
  ("5", "3", "friend"),
  ("3", "6", "friend"),
  ("6", "3", "friend"),
  ("3", "7", "friend"),
  ("7", "3", "friend")
], ["src", "dst", "relationship"])

g = GraphFrame(vertices, edges)


In [4]:
g.vertices.filter("age > 30")

DataFrame[id: string, name: string, age: bigint, gender: string, university: string]

In [5]:
g.inDegrees.filter("inDegree > 2").show()

+---+--------+
| id|inDegree|
+---+--------+
|  1|       3|
|  2|       3|
|  3|       6|
+---+--------+



In [6]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  1|       3|
|  2|       3|
|  3|       6|
|  4|       2|
|  5|       2|
|  6|       1|
|  7|       1|
+---+--------+



In [7]:
g.find("(a)-[e]->(b); (b)-[e2]->(a)").show(15)

+------------+--------------------+--------------------+------------+
|           e|                   a|                   b|          e2|
+------------+--------------------+--------------------+------------+
|[1,4,friend]|  [1,Alex,28,M,MIPT]| [4,Pavel,30,M,MIPT]|[4,1,friend]|
|[3,4,friend]|[3,Natasha,27,F,S...| [4,Pavel,30,M,MIPT]|[4,3,friend]|
|[2,5,friend]| [2,Emeli,28,F,MIPT]|  [5,Oleg,35,M,MIPT]|[5,2,friend]|
|[3,5,friend]|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|[5,3,friend]|
|[3,6,friend]|[3,Natasha,27,F,S...|   [6,Ivan,30,M,MSU]|[6,3,friend]|
|[2,1,friend]| [2,Emeli,28,F,MIPT]|  [1,Alex,28,M,MIPT]|[1,2,friend]|
|[3,1,friend]|[3,Natasha,27,F,S...|  [1,Alex,28,M,MIPT]|[1,3,friend]|
|[4,1,friend]| [4,Pavel,30,M,MIPT]|  [1,Alex,28,M,MIPT]|[1,4,friend]|
|[3,7,friend]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|[7,3,friend]|
|[1,2,friend]|  [1,Alex,28,M,MIPT]| [2,Emeli,28,F,MIPT]|[2,1,friend]|
|[3,2,friend]|[3,Natasha,27,F,S...| [2,Emeli,28,F,MIPT]|[2,3,friend]|
|[5,2,friend]|  [5,O

In [8]:
motifs = g.find("(A)-[]->(B); (B)-[]->(C)")
motifs.show(15)

+--------------------+--------------------+-------------------+
|                   A|                   B|                  C|
+--------------------+--------------------+-------------------+
|  [1,Alex,28,M,MIPT]| [2,Emeli,28,F,MIPT]| [1,Alex,28,M,MIPT]|
|[3,Natasha,27,F,S...| [2,Emeli,28,F,MIPT]| [1,Alex,28,M,MIPT]|
|  [5,Oleg,35,M,MIPT]| [2,Emeli,28,F,MIPT]| [1,Alex,28,M,MIPT]|
|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|  [1,Alex,28,M,MIPT]| [4,Pavel,30,M,MIPT]| [1,Alex,28,M,MIPT]|
|[3,Natasha,27,F,S...| [4,Pavel,30,M,MIPT]| [1,Alex,28,M,MIPT]|
| [2,Emeli,28,F,MIPT]|  [1,Alex,28,M,MIPT]|[2,Emeli,28,F,MIPT]|
|[3,Natasha,27,F,S...|  [1,Alex,28,M,MIP

In [9]:
motifs = g.find("(A)-[]->(B); (B)-[]->(C)").filter("A.id != C.id")
motifs.show(15)

+--------------------+--------------------+-------------------+
|                   A|                   B|                  C|
+--------------------+--------------------+-------------------+
|[3,Natasha,27,F,S...| [2,Emeli,28,F,MIPT]| [1,Alex,28,M,MIPT]|
|  [5,Oleg,35,M,MIPT]| [2,Emeli,28,F,MIPT]| [1,Alex,28,M,MIPT]|
| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...| [1,Alex,28,M,MIPT]|
|[3,Natasha,27,F,S...| [4,Pavel,30,M,MIPT]| [1,Alex,28,M,MIPT]|
|[3,Natasha,27,F,S...|  [1,Alex,28,M,MIPT]|[2,Emeli,28,F,MIPT]|
| [4,Pavel,30,M,MIPT]|  [1,Alex,28,M,MIPT]|[2,Emeli,28,F,MIPT]|
|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|[2,Emeli,28,F,MIPT]|
| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|[2,Emeli,28,F,MIPT]|
|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S.

In [10]:
AC = motifs.selectExpr("A.id as A", "C.id as C")
AC.show(15)

+---+---+
|  A|  C|
+---+---+
|  3|  1|
|  5|  1|
|  2|  1|
|  4|  1|
|  5|  1|
|  6|  1|
|  7|  1|
|  3|  1|
|  3|  2|
|  4|  2|
|  1|  2|
|  4|  2|
|  5|  2|
|  6|  2|
|  7|  2|
+---+---+
only showing top 15 rows



In [11]:
AC.groupBy("A", "C").count().filter("A = 1").show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  1|  2|    1|
|  1|  3|    2|
|  1|  4|    1|
|  1|  5|    2|
|  1|  6|    1|
|  1|  7|    1|
+---+---+-----+



In [14]:
AC.groupBy("A", "C").count().show(15)

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  4|  1|    1|
|  4|  2|    2|
|  4|  3|    1|
|  4|  4|    2|
|  4|  5|    1|
|  4|  6|    1|
|  4|  7|    1|
|  5|  1|    2|
|  5|  2|    1|
|  5|  3|    1|
|  5|  4|    1|
|  5|  5|    2|
|  5|  6|    1|
|  5|  7|    1|
|  6|  1|    1|
+---+---+-----+
only showing top 15 rows



## lesson 3.3

In [12]:
results = g.triangleCount()
results.show()

+-----+---+-------+---+------+----------+
|count| id|   name|age|gender|university|
+-----+---+-------+---+------+----------+
|    2|  1|   Alex| 28|     M|      MIPT|
|    2|  2|  Emeli| 28|     F|      MIPT|
|    3|  3|Natasha| 27|     F|     SPbSU|
|    1|  4|  Pavel| 30|     M|      MIPT|
|    1|  5|   Oleg| 35|     M|      MIPT|
|    0|  6|   Ivan| 30|     M|       MSU|
|    0|  7|   Ilya| 29|     M|       MSU|
+-----+---+-------+---+------+----------+



In [ ]:
edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

graphData = sc.parallelize(edgeList).map(lambda (src, dst): Row(src, dst))

graphSchemaAB = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('B', StringType(), nullable=False)])

abDF = sqlCtx.createDataFrame(graphData, graphSchemaAB)

abDF.show()

In [ ]:
graphSchemaBC1 = StructType([StructField('B', IntegerType(), nullable=False),
                       StructField('C1', StringType(), nullable=False)])
bc1DF = sqlCtx.createDataFrame(graphData, graphSchemaBC1)

graphSchemaAC2 = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('C2', StringType(), nullable=False)])
ac2DF = sqlCtx.createDataFrame(graphData, graphSchemaAC2)

In [ ]:
bc1DF.show()

In [ ]:
ac2DF.show()

In [ ]:
abc1DF = abDF.join(bc1DF, "B")
abc1DF.show()

In [ ]:
abc1c2DF = abc1DF.join(ac2DF, "A")
abc1c2DF.show()

In [ ]:
abc1c2DF.filter("C1 = C2").show()

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
{
    abc1c2DF.filter("C1 = C2")
    .select(array(col("A"), col("B"), col("C1")).alias("triangleVertices"))
    .select(explode("triangleVertices").alias("triangleVertex"))
    .groupBy("triangleVertex")
    .count()
    .show()
}

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

graphData = sc.parallelize(edgeList).map(lambda (src, dst): Row(src, dst))

graphSchemaAB = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('B', StringType(), nullable=False)])

abDF = sqlCtx.createDataFrame(graphData, graphSchemaAB)

graphSchemaBC1 = StructType([StructField('B', IntegerType(), nullable=False),
                       StructField('C1', StringType(), nullable=False)])
bc1DF = sqlCtx.createDataFrame(graphData, graphSchemaBC1)

graphSchemaAC2 = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('C2', StringType(), nullable=False)])
ac2DF = sqlCtx.createDataFrame(graphData, graphSchemaAC2)

abc1DF = abDF.join(bc1DF, "B")
abc1c2DF = abc1DF.join(ac2DF, "A")


vertexTriangle = abc1c2DF.filter("C1 = C2")
    .select(array(col("A"), col("B"), col("C1")).alias("triangleVertices"))
    .select(explode("triangleVertices").alias("triangleVertex"))
    .groupBy("triangleVertex")
    .count()
    .show()
    

In [ ]:
edgeListBC = sc.broadcast(set(edgeList))

# Define udf
from pyspark.sql.functions import udf

def isInEdgeList(src, dst):
    return (int(src), int(dst)) in edgeListBC.value

udfIsInEdgeList = udf(isInEdgeList, StringType())

abc1DF.withColumn("isTriangle", udfIsInEdgeList("A", "C1")).show()

In [13]:
from graphframes import * 

vertices = sqlCtx.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("1", "3", "friend"),
  ("1", "4", "friend"),
  ("2", "3", "friend"),
  ("2", "5", "friend"),
  ("3", "4", "friend"),
  ("3", "5", "friend"),
  ("3", "6", "friend"),
  ("3", "7", "friend")
], ["src", "dst", "type"])

g = GraphFrame(vertices, edges)

In [14]:
triangles = g.find("(A)-[]->(B); (B)-[]->(C); (A)-[]->(C)")

In [15]:
triangles.show()

+-------------------+--------------------+--------------------+
|                  A|                   B|                   C|
+-------------------+--------------------+--------------------+
| [1,Alex,28,M,MIPT]| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|
| [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...| [4,Pavel,30,M,MIPT]|
|[2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|
+-------------------+--------------------+--------------------+



In [18]:
vertexTriangles = triangles.selectExpr("A.id as A", "B.id as B", "C.id as C") \
    .select(array(col("A"), col("B"), col("C")).alias("triangleVertices")) \
    .select(explode("triangleVertices").alias("id")) \
    .groupBy("id") \
    .count()

In [19]:
vertexTriangles.show()

+---+-----+
| id|count|
+---+-----+
|  1|    2|
|  2|    2|
|  3|    3|
|  4|    1|
|  5|    1|
+---+-----+



In [20]:
g.vertices.join(vertexTriangles, "id", "left_outer").show()

+---+-------+---+------+----------+-----+
| id|   name|age|gender|university|count|
+---+-------+---+------+----------+-----+
|  1|   Alex| 28|     M|      MIPT|    2|
|  2|  Emeli| 28|     F|      MIPT|    2|
|  3|Natasha| 27|     F|     SPbSU|    3|
|  4|  Pavel| 30|     M|      MIPT|    1|
|  5|   Oleg| 35|     M|      MIPT|    1|
|  6|   Ivan| 30|     M|       MSU| null|
|  7|   Ilya| 29|     M|       MSU| null|
+---+-------+---+------+----------+-----+

